# Importing packages

In [39]:
import pandas as pd
import numpy as np
import openpyxl
import os
import requests
from helpers import data_folder #function to check for data folder and create it if not there
from helpers import data_checker #function to check for a specific file inside the data folder
import json
import folium
import zipfile

data_folder()

Data checker will look for the data folder. If none is found then it will create it.
Successfully created the directory Data 


# Importing Index of Deprivation Data
These data are published separately for England, Scotland, Wales and Northern Ireland.

The aim is to get IMD rank, but also the decile. IMD rank is nation specific, so cannot be used to compare places in Scotland. But within a council area, it will be a valid indicator. Decile may be the more useful thing, though. At this stage unsure, so keep both.

## First, download the English data

In [40]:
if not data_checker('england_imd_2019.xlsx'):
    #this url links to the MHCLG website that hosts the table on English IMD by LSOA. The data come in an xlsx file.
    url = 'https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/833970/File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx'
    r = requests.get(url, allow_redirects = True)
    with open (os.path.join('Data', 'england_imd_2019.xlsx'), 'wb') as f:
        f.write(r.content)

Data has already been downloaded


In [41]:
#import data into pandas dataframe
df = pd.read_excel(os.path.join('Data', 'england_imd_2019.xlsx'), sheet_name = 'IMD2019')
df.shape

(32844, 6)

In [87]:
df.head()

,lsoa_code,int_zone,Council_area,total_pop,adult_pop,imd_rank,imd_decile
0,S01006506,Culter,Aberdeen City,894,580,4691,4
1,S01006507,Culter,Aberdeen City,793,470,4862,4
2,S01006508,Culter,Aberdeen City,624,461,5686,2
3,S01006509,Culter,Aberdeen City,537,307,4332,4
4,S01006510,Culter,Aberdeen City,663,415,3913,5


### Do a little clean up
Tidy column titles, etc...

In [88]:
#first, change column titles to something a bit better...
df = df.rename(columns = {'LSOA code (2011)': 'lsoa_code', 'LSOA name (2011)': 'lsoa_name_211', 'Local Authority District code (2019)': 'local_authority_district_code', 'Local Authority District name (2019)': 'local_authority', 'Index of Multiple Deprivation (IMD) Rank': 'imd_rank', 'Index of Multiple Deprivation (IMD) Decile': 'imd_decile'})
#df.head()

#### Finding english population data
The Scottish IMD data has population, so let's download and add this to the english data

In [100]:
if not data_checker('eng_pop.xlsx'):
    print('Downloading English population data')
    #This is data for the whole of the UK:
    lookup_url = 'https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2flowersuperoutputareamidyearpopulationestimates%2fmid2019sape22dt2/sape22dt2mid2019lsoasyoaestimatesunformatted.zip' 
    r = requests.get(lookup_url, allow_redirects=True)
    
    #first, create the path destination for the downloaded zip file
    zip_dir = os.path.join('Data', 'eng_pop.zip')
    
    #now create the zip file and write the content of our requests object (i.e. the zip file)
    with open(zip_dir, 'wb') as zip_f: 
        zip_f.write(r.content)
    print('File downloaded, now extracting data...')
    
    #extract the csv file from the zip archive
    with zipfile.ZipFile(zip_dir, 'r') as zip_f: 
        zip_f.extract('SAPE22DT2-mid-2019-lsoa-syoa-estimates-unformatted.xlsx', path = 'Data')
        
    #get rid of unneeded zip file
    os.remove(zip_dir) 
    
    #rename unnecessarily long csv name to 'postcodes'csv:
    os.rename(os.path.join('Data', 'SAPE22DT2-mid-2019-lsoa-syoa-estimates-unformatted.xlsx'), os.path.join('Data', 'eng_pop.xlsx')) 
    print('Done, file is saved as data/eng_pop.xlsx')
else:
    print('eng_pop.xlsx already exists in the data file, no need to download it!')

Data has already been downloaded
eng_pop.xlsx already exists in the data file, no need to download it!


In [108]:
df_pop = pd.read_excel(os.path.join('Data', 'eng_pop.xlsx'), sheet_name = 'Mid-2019 Persons')

C:\Users\Ben\.conda\envs\local_auth_env\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [109]:
df_pop_backup #is the backup file from the above, to prevent repeated slow importing

In [123]:
df_pop = df_pop_backup
df_pop = df_pop.drop([0, 1, 2])
df_pop.head()

,Contents,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97
3,LSOA Code,LSOA Name,LA Code (2019 boundaries),LA name (2019 boundaries),LA Code (2020 boundaries),LA name (2020 boundaries),All Ages,0.0,1.0,2.0,...,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90+
4,E01011949,Hartlepool 009A,E06000001,Hartlepool,E06000001,Hartlepool,1954,27.0,22.0,13.0,...,12.0,15.0,9.0,7.0,10.0,10.0,5.0,3.0,0.0,10
5,E01011950,Hartlepool 008A,E06000001,Hartlepool,E06000001,Hartlepool,1257,12.0,9.0,7.0,...,4.0,2.0,1.0,3.0,1.0,2.0,3.0,2.0,1.0,4
6,E01011951,Hartlepool 007A,E06000001,Hartlepool,E06000001,Hartlepool,1209,18.0,9.0,5.0,...,6.0,2.0,6.0,3.0,4.0,3.0,2.0,4.0,1.0,3
7,E01011952,Hartlepool 002A,E06000001,Hartlepool,E06000001,Hartlepool,1740,25.0,30.0,12.0,...,16.0,12.0,7.0,15.0,16.0,14.0,11.0,15.0,14.0,67


In [127]:
#reset header
new_header = df_pop.iloc[0]
df_pop = df_pop[1:].reset_index()
df_pop.columns = new_header

AttributeError: 'function' object has no attribute 'iloc'

In [128]:
df_pop.head()

AttributeError: 'function' object has no attribute 'head'

In [62]:
#now save to a nation-specific variable
eng_df = df
eng_df.head()

,Data_Zone,Intermediate_Zone,Council_area,Total_population,Working_age_population,SIMD2020v2_Rank
0,S01006506,Culter,Aberdeen City,894,580,4691
1,S01006507,Culter,Aberdeen City,793,470,4862
2,S01006508,Culter,Aberdeen City,624,461,5686
3,S01006509,Culter,Aberdeen City,537,307,4332
4,S01006510,Culter,Aberdeen City,663,415,3913


## Now, download the Scottish data

In [80]:
if not data_checker('scotland_imd_2020.xlsx'):
    print('No Scottish deprivation data found, downloading...')
    url = 'https://www.gov.scot/binaries/content/documents/govscot/publications/statistics/2020/01/scottish-index-of-multiple-deprivation-2020-ranks-and-domain-ranks/documents/scottish-index-of-multiple-deprivation-2020-ranks-and-domain-ranks/scottish-index-of-multiple-deprivation-2020-ranks-and-domain-ranks/govscot%3Adocument/SIMD%2B2020v2%2B-%2Branks.xlsx'
    r = requests.get(url)
    with open (os.path.join('Data', 'scotland_imd_2020.xlsx'), 'wb') as f:
        f.write(r.content)

Data has already been downloaded


In [81]:
#import data into a pandas dataframe
df = pd.read_excel(os.path.join('Data', 'scotland_imd_2020.xlsx'), sheet_name = 'SIMD 2020v2 ranks')
df.head()

,Data_Zone,Intermediate_Zone,Council_area,Total_population,Working_age_population,SIMD2020v2_Rank,SIMD2020v2_Income_Domain_Rank,SIMD2020_Employment_Domain_Rank,SIMD2020_Health_Domain_Rank,SIMD2020_Education_Domain_Rank,SIMD2020_Access_Domain_Rank,SIMD2020_Crime_Domain_Rank,SIMD2020_Housing_Domain_Rank
0,S01006506,Culter,Aberdeen City,894,580,4691,3936.0,3220.0,5174,5887,4724,4664.0,3248.0
1,S01006507,Culter,Aberdeen City,793,470,4862,4829.0,4481.0,5051,4384,2148,4602.0,3486.0
2,S01006508,Culter,Aberdeen City,624,461,5686,4460.0,5110.0,5942,5915,4200,4563.5,5342.0
3,S01006509,Culter,Aberdeen City,537,307,4332,3481.0,3229.0,3871,6401,3982,5626.0,4394.5
4,S01006510,Culter,Aberdeen City,663,415,3913,3344.0,3448.0,3049,4092,5588,3885.0,3736.0


### Do a little clean up
Drop columns other than rank. Notice there's no decile, so will have to create one.

In [82]:
df = df.drop(columns = ['SIMD2020v2_Income_Domain_Rank', 'SIMD2020_Employment_Domain_Rank', 'SIMD2020_Health_Domain_Rank', 'SIMD2020_Education_Domain_Rank', 'SIMD2020_Access_Domain_Rank', 'SIMD2020_Crime_Domain_Rank', 'SIMD2020_Housing_Domain_Rank'])
#rename columns something easier
df = df.rename(columns={'Data_Zone': 'lsoa_code', 'Intermediate_Zone': 'int_zone', 'Council_Area': 'local_authority', 'Total_population': 'total_pop', 'Working_age_population': 'adult_pop', 'SIMD2020v2_Rank': 'imd_rank'})

In [84]:
#create a decile column
df['imd_decile'] = pd.qcut(df['imd_rank'], 10, labels = np.arange(10, 0, -1))

In [86]:
#lastly, give it a more distinctive name:
scot_df = df
scot_df.head()

,lsoa_code,int_zone,Council_area,total_pop,adult_pop,imd_rank,imd_decile
0,S01006506,Culter,Aberdeen City,894,580,4691,4
1,S01006507,Culter,Aberdeen City,793,470,4862,4
2,S01006508,Culter,Aberdeen City,624,461,5686,2
3,S01006509,Culter,Aberdeen City,537,307,4332,4
4,S01006510,Culter,Aberdeen City,663,415,3913,5


## Download LSOA to Postcode lookup (additional location labels)
Use data_checker to check whether postcodes.csv is already in the data file. If not, then script will download and extract a lookup csv spreadsheet from the UK government geoportal containing postcodes to LSOA to MSOA info. This will allow us to join data gathered on households to particular areas, and facilitate further analysis and mapping.

In [46]:
if not data_checker('postcodes.csv'):
    print('Downloading LSOA to postcode lookup...')
    #This is data for the whole of the UK:
    lookup_url = 'https://www.arcgis.com/sharing/rest/content/items/940cf89bf08e4459bdf0470bbd345424/data' 
    r = requests.get(lookup_url, allow_redirects=True)
    
    #first, create the path destination for the downloaded zip file
    zip_dir = os.path.join('Data', 'postcodes.zip') 
    
    #now create the zip file and write the content of our requests object (i.e. the zip file)
    with open(zip_dir, 'wb') as zip_f: 
        zip_f.write(r.content)
    print('File downloaded, now extracting data...')
    
    #extract the csv file from the zip archive
    with zipfile.ZipFile(zip_dir, 'r') as zip_f: 
        zip_f.extract('PCD_OA_LSOA_MSOA_LAD_NOV19_UK_LU.csv', path = 'Data')
        
    #get rid of unneeded zip file
    os.remove(zip_dir) 
    
    #rename unnecessarily long csv name to 'postcodes'csv:
    os.rename(os.path.join('Data', 'PCD_OA_LSOA_MSOA_LAD_NOV19_UK_LU.csv'), os.path.join('Data', 'postcodes.csv')) 
    print('Done, file is saved as data/postcodes.csv')
else:
    print('Postcodes.csv already exists in the data file, no need to download it!')

Data has already been downloaded
Postcodes.csv already exists in the data file, no need to download it!


## Now join English IMD data to additional location labels

## Importing Geographical data and displaying in folium

In [47]:
m = folium.Map(location = [55.09, -1.5], zoom_start = 5) #pretty much centres it on the UK
#m #uncomment to view starting map


In [48]:
url = 'https://opendata.arcgis.com/datasets/c892586698ad4d268f9288f1df20ab77_0.geojson'

folium.TopoJson(json.loads(requests.get(url).text), 'objects.ana')